In [26]:
from IPython.display import display, Javascript

def restart_kernel():
    display(Javascript('IPython.notebook.kernel.restart()'))

restart_kernel()

<IPython.core.display.Javascript object>

In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Conv1D, dot, Dropout
from keras.models import Model
from keras.layers import BatchNormalization  # Modificación realizada aquí
from keras.layers import MaxPooling2D, MaxPooling1D
from keras.layers import Concatenate
from keras.layers import Lambda, Flatten, Dense, Dropout
from keras.initializers import glorot_uniform
from keras.layers import Layer
from keras.regularizers import l2
from keras import backend as K
from sklearn.metrics import accuracy_score, roc_auc_score

In [28]:
import sys
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import os, random, pickle
from sklearn.model_selection import KFold
import numpy.random as rng
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [29]:
seed = 2556
np.random.seed(seed)

In [30]:
def initialize_weights(shape, name=None, dtype=None):
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

def initialize_bias(shape, name=None, dtype=None):
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [31]:
def last_layer(encoded_l, encoded_r, lyr_name='cos'):

    if lyr_name == 'L1':
        # Add a customized layer to compute the absolute difference between the encodings
        L1_layer = Lambda(lambda tensors: tf.keras.backend.abs(tensors[0] - tensors[1]))
        L1_distance = L1_layer([encoded_l, encoded_r])
        add_dens =  Dense(512, activation='relu', bias_initializer=initialize_bias)(L1_distance)
        # drp_lyr = Dropout(0.25)(add_dens)
        # xx =  Dense(128, activation='relu', bias_initializer=initialize_bias)(add_dens)
        # prediction = Dense(1, activation='sigmoid', bias_initializer=initialize_bias)(L1_distance)
        prediction = Dense(1, activation='sigmoid')(xx)

    elif lyr_name == 'L2':

        # Write L2 here
        L2_layer = Lambda(lambda tensors: (tensors[0] - tensors[1]) ** 2 / (tensors[0] + tensors[1]))
        L2_distance = L2_layer([encoded_l, encoded_r])
        add_dens = Dense(512, activation='relu', bias_initializer=initialize_bias)(L2_distance)
        drp_lyr = Dropout(0.25)(add_dens)
        # xx =  Dense(128, activation='relu', bias_initializer=initialize_bias)(drp_lyr)
        # drp_lyr2 = Dropout(0.25)(xx)
        # x =  Dense(64, activation='relu', bias_initializer=initialize_bias)(xx)
        prediction = Dense(1, activation='sigmoid', bias_initializer=initialize_bias)(drp_lyr)

    else:

        # Add cosine similarity function
        cos_layer= Lambda(lambda tensors: K.sum(tensors[0] * tensors[1], axis=-1, keepdims=True)/
                                          tf.keras.backend.l2_normalize(tensors[0])*tf.keras.backend.l2_normalize(tensors[1]))
        cos_distance = cos_layer([encoded_l, encoded_r])
        prediction = Dense(1, activation='sigmoid', bias_initializer=initialize_bias)(cos_distance)

    return prediction

In [32]:
def get_test_pairs(sample_size=1000):
    # Inicializar listas para almacenar los pares y las etiquetas
    pairs = [[], []]
    targets = []

    # Generar índices aleatorios para los pares
    indices = np.random.choice(len(y_test), size=(sample_size, 2), replace=True)

    for idx1, idx2 in indices:
        pairs[0].append(x_test.iloc[idx1].values.reshape(genes_len, 1))
        pairs[1].append(x_test.iloc[idx2].values.reshape(genes_len, 1))
        targets.append(int(y_test[idx1] == y_test[idx2]))

    # Convertir las listas en arrays numpy
    pairs[0] = np.array(pairs[0])
    pairs[1] = np.array(pairs[1])
    targets = np.array(targets)

    return pairs, targets


In [33]:
def get_siamese_model(input_shape):
    # Define the inputs for the two samples
    left_input = Input(shape=input_shape)
    right_input = Input(shape=input_shape)

    # Shared Dense network with Flatten layer
    shared_dense = Sequential([
        Flatten(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='sigmoid')
    ])

    # Generate the encodings for the two inputs
    encoded_l = shared_dense(left_input)
    encoded_r = shared_dense(right_input)

    # Compute the absolute difference between the encodings
    L1_distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([encoded_l, encoded_r])

    # Add a final Dense layer with sigmoid activation
    prediction = Dense(1, activation='sigmoid')(L1_distance)

    # Define the Siamese network model
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
    return siamese_net




In [34]:
"""
def get_siamese_model(input_shape):
    # Definir las entradas para las dos muestras
    left_input = Input(shape=input_shape)
    right_input = Input(shape=input_shape)
    
    # Red Neuronal Simple
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))

    # Generar las representaciones (embeddings) para las dos entradas
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Cálculo de la distancia L1 entre las representaciones
    L1_distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([encoded_l, encoded_r])
    
    # Capa de decisión final
    prediction = Dense(1, activation='sigmoid')(L1_distance)
    
    # Definir el modelo
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
    return siamese_net
"""

"\ndef get_siamese_model(input_shape):\n    # Definir las entradas para las dos muestras\n    left_input = Input(shape=input_shape)\n    right_input = Input(shape=input_shape)\n    \n    # Red Neuronal Simple\n    model = Sequential()\n    model.add(Flatten(input_shape=input_shape))\n    model.add(Dense(64, activation='relu'))\n    model.add(Dense(32, activation='relu'))\n    model.add(Dense(16, activation='relu'))\n\n    # Generar las representaciones (embeddings) para las dos entradas\n    encoded_l = model(left_input)\n    encoded_r = model(right_input)\n    \n    # Cálculo de la distancia L1 entre las representaciones\n    L1_distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([encoded_l, encoded_r])\n    \n    # Capa de decisión final\n    prediction = Dense(1, activation='sigmoid')(L1_distance)\n    \n    # Definir el modelo\n    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)\n    return siamese_net\n"

In [35]:
from sklearn.metrics import accuracy_score, roc_auc_score

def evaluate_model(model):
    pairs_test, targets_test = get_test_pairs()
    predictions = model.predict(pairs_test)
    predictions = predictions.ravel()

    # Calcular métricas
    acc = accuracy_score(targets_test, predictions > 0.5)
    roc_auc = roc_auc_score(targets_test, predictions)

    print(f"Precisión en el conjunto de prueba: {acc:.2f}")
    print(f"ROC-AUC en el conjunto de prueba: {roc_auc:.2f}")


In [36]:
#comentario
"""
def get_siamese_model(input_shape):
    # Definir las entradas para las dos muestras
    left_input = Input(input_shape)
    right_input = Input(input_shape)

    # Red Neuronal Convolucional Compartida
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='sigmoid'))

    # Generar las representaciones (embeddings) para las dos entradas
    encoded_l = model(left_input)
    encoded_r = model(right_input)

    # Cálculo de la distancia L1 entre las representaciones
    L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])

    # Capa de decisión final
    prediction = Dense(1, activation='sigmoid')(L1_distance)

    # Definir el modelo
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
    return siamese_net"""


"\ndef get_siamese_model(input_shape):\n    # Definir las entradas para las dos muestras\n    left_input = Input(input_shape)\n    right_input = Input(input_shape)\n\n    # Red Neuronal Convolucional Compartida\n    model = Sequential()\n    model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=input_shape))\n    model.add(MaxPooling1D(pool_size=2))\n    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))\n    model.add(MaxPooling1D(pool_size=2))\n    model.add(Flatten())\n    model.add(Dense(128, activation='sigmoid'))\n\n    # Generar las representaciones (embeddings) para las dos entradas\n    encoded_l = model(left_input)\n    encoded_r = model(right_input)\n\n    # Cálculo de la distancia L1 entre las representaciones\n    L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))\n    L1_distance = L1_layer([encoded_l, encoded_r])\n\n    # Capa de decisión final\n    prediction = Dense(1, activation='sigmoid')(L1_distance)\n\n    # Defini

In [37]:
data_path = "tcga_common_data_peru.csv"
TCGA_data = pd.read_csv(data_path)

In [38]:
TCGA_data.head()

,OR4F5,SAMD11,KLHL17,PLEKHN1,ISG15,AGRN,TTLL10,B3GALT6,SCNN1D,PUSL1,...,UTY,VCY,CDY2B,HSFY2,KDM5D,RBMY1E,PRY2,CDY1B,DAZ3,labels
0,0.254728,3.199541,2.841540,1.753755,3.672659,3.421789,0.358315,3.293547,2.954878,3.032272,...,0.201748,0.297927,0.227856,0.168309,0.198074,0.224009,0.312519,0.202187,0.225862,cancer
1,0.254728,3.016719,2.546730,1.320451,3.441749,3.403565,0.358315,3.091509,2.645649,2.890673,...,2.971343,0.297927,0.227856,0.168309,3.204342,0.224009,0.312519,0.202187,0.225862,healthy
2,0.254728,2.029367,2.708073,0.668227,3.139925,3.555979,0.358315,2.988379,2.381621,2.917346,...,2.885337,0.297927,0.227856,0.168309,2.765418,0.224009,0.312519,0.202187,0.225862,cancer
3,0.254728,0.244785,2.733089,1.884489,3.260723,3.374143,0.358315,3.157854,2.823415,3.011421,...,0.201748,0.297927,0.227856,0.168309,0.198074,0.224009,0.312519,0.202187,0.225862,cancer
4,0.254728,2.470030,2.674949,2.086111,3.183263,3.496209,0.358315,3.091784,2.899107,2.797831,...,2.601769,0.297927,0.227856,0.168309,2.904073,0.224009,0.312519,0.202187,0.225862,cancer


In [39]:
#selecciona datos de rna_seq_data_common donde labels es cancer
TCGA_data__cancer = TCGA_data[TCGA_data.labels == 'cancer'].sample(100, random_state=seed)
#selecciona 50 datos de rna_seq_data_common donde labels es healthy
TCGA_data_healthy = TCGA_data[TCGA_data.labels == 'healthy'].sample(50, random_state=seed)
#concatena los datos
TCGA_data_balanced = pd.concat([TCGA_data__cancer, TCGA_data_healthy])

In [40]:
#y = TCGA_data_balanced['labels'].values
y = np.where(TCGA_data_balanced['labels'].values == 'cancer', 1, 0)
X = TCGA_data_balanced.drop('labels', axis=1).values

In [41]:
genes_len = X.shape[1]
input_shape = (genes_len, 1)

In [42]:
from sklearn.model_selection import StratifiedKFold

# Configurar K-Fold Cross-Validation
n_splits = 10
#kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [43]:
model_path = "/data/sespinoza/objetivo1/CancerSiamese/"

In [44]:
# Hiperparámetros
batch_size = 64
n_iter = 2000  # Puedes ajustar este número
evaluate_every = 100  # Frecuencia de evaluación
N_way = 1
n_val = 100  # Número de tareas de one-shot para validar
best_acc = -1

In [45]:
X.shape

(150, 16075)

In [46]:
#cuenta la cantidad de valores por clase en y
unique, counts = np.unique(y, return_counts=True)
#impresión de la cantidad de valores por clase
print(np.asarray((unique, counts)).T)

[[  0  50]
 [  1 100]]


In [47]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [48]:
#import tensorflow as tf
#tf.config.optimizer.set_jit(False)  # Desactiva la compilación JIT

In [49]:
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import SGD
fold = 1
for train_index, test_index in skf.split(X, y):
    print(f"\nEntrenando el pliegue {fold}/{n_splits}")

    # Dividir los datos en entrenamiento y prueba para este pliegue
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Verificar la presencia de ambas clases en y_train
    unique_classes = np.unique(y_train)
    if len(unique_classes) < 2:
        print(f"Advertencia: Sólo una clase presente en el pliegue {fold}. Saltando este pliegue.")
        continue  # Salta al siguiente pliegue

    # Convertir los datos a DataFrames para mantener consistencia con el código original
    X_train_df = pd.DataFrame(X_train)
    X_test_df = pd.DataFrame(X_test)

    # Calcular MI en el conjunto de entrenamiento
    MI = mutual_info_classif(X_train_df, y_train)
    n_features_mi = 5000  # Puedes ajustar este número
    selected_scores_indices = np.argsort(MI)[::-1][:n_features_mi]

    # Seleccionar las características en base a MI
    X_train_mi = X_train_df.iloc[:, selected_scores_indices]
    X_test_mi = X_test_df.iloc[:, selected_scores_indices]

    # **Paso 2: Selección de Características con RFE**
    estimator = linear_model.LogisticRegression(max_iter=700, solver='liblinear')
    n_features_rfe = 25  # Puedes ajustar este número
    rfe = RFE(estimator=estimator, n_features_to_select=n_features_rfe, step=1)
    rfe.fit(X_train_mi, y_train)

    # Transformar los datos de entrenamiento y prueba
    X_train_final = rfe.transform(X_train_mi)
    X_test_final = rfe.transform(X_test_mi)

    print('NaN en X_train_final:', np.isnan(X_train_final).any())
    print('Inf en X_train_final:', np.isinf(X_train_final).any())
    
    scaler = StandardScaler()
    X_train_final = scaler.fit_transform(X_train_final)
    X_test_final = scaler.transform(X_test_final)

    X_train_final = X_train_final.astype('float32')
    X_test_final = X_test_final.astype('float32')

    # Actualizar genes_len e input_shape
    genes_len = X_train_final.shape[1]
    input_shape = (genes_len, 1)

    x_train = pd.DataFrame(X_train_final)
    x_test = pd.DataFrame(X_test_final)

    # Definir el modelo
    model = get_siamese_model(input_shape)
    # Update the learning rate to a reasonable value
    optimizer = Adam(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Definir las funciones get_batch y get_test_pairs utilizando los datos del pliegue actual

    def get_batch(batch_size):
        # Función para generar un lote de entrenamiento
        half_batch = batch_size // 2
        pairs = [np.zeros((batch_size, genes_len, 1)) for _ in range(2)]
        targets = np.zeros((batch_size,))
        targets[half_batch:] = 1  # La segunda mitad son pares de la misma clase

        # Obtener índices de muestras de cada clase
        cancer_indices = np.where(y_train == 1)[0]
        non_cancer_indices = np.where(y_train == 0)[0]

        for i in range(half_batch):
            # Pares de clases diferentes (una cáncer, otra no cáncer)
            idx_cancer = np.random.choice(cancer_indices)
            idx_non_cancer = np.random.choice(non_cancer_indices)

            pairs[0][i, :, 0] = x_train.iloc[idx_cancer].values
            pairs[1][i, :, 0] = x_train.iloc[idx_non_cancer].values

        for i in range(half_batch, batch_size):
            # Pares de la misma clase
            if np.random.rand() < 0.5:
                # Ambos cáncer
                if len(cancer_indices) >= 2:
                    idx1, idx2 = np.random.choice(cancer_indices, size=2, replace=False)
                else:
                    idx1 = idx2 = np.random.choice(cancer_indices)
            else:
                # Ambos no cáncer
                if len(non_cancer_indices) >= 2:
                    idx1, idx2 = np.random.choice(non_cancer_indices, size=2, replace=False)
                else:
                    idx1 = idx2 = np.random.choice(non_cancer_indices)

            pairs[0][i, :, 0] = x_train.iloc[idx1].values
            pairs[1][i, :, 0] = x_train.iloc[idx2].values

        return pairs, targets

    # Definir las funciones get_test_pairs y evaluate_model como antes...

    # Entrenamiento del modelo
    n_iter = 2000  # Ajusta este número según tus recursos
    batch_size = 64
    evaluate_every = 100
    best_acc = -1

    for i in range(1, n_iter + 1):
        inputs, targets = get_batch(batch_size)
        print('Shape of inputs[0]:', inputs[0].shape)
        print('Shape of inputs[1]:', inputs[1].shape)
        print('Shape of targets:', targets.shape)
        loss, acc = model.train_on_batch(inputs, targets)
        

        if i % evaluate_every == 0:
            print(f"Iteración {i}, pérdida de entrenamiento: {loss:.4f}, precisión: {acc:.4f}")
            evaluate_model(model)

    fold += 1


Entrenando el pliegue 1/10
NaN en X_train_final: False
Inf en X_train_final: False
Shape of inputs[0]: (64, 25, 1)
Shape of inputs[1]: (64, 25, 1)
Shape of targets: (64,)


AttributeError: 'SGD' object has no attribute 'get_updates'

In [14]:
fold=1
for train_index, test_index in kf.split(X):
    print(f"\nEntrenando el pliegue {fold}/{n_splits}")

    # Dividir los datos en entrenamiento y prueba para este pliegue
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convertir los datos a DataFrames para mantener consistencia con el código original
    x_train = pd.DataFrame(X_train, index=y_train)
    x_test = pd.DataFrame(X_test, index=y_test)

    # Crear diccionarios de índices por clase
    def indices_save(labels):
        class_dic = {}
        for k in np.unique(labels):
            indices = [i for i, label in enumerate(labels) if label == k]
            class_dic[k] = indices
        return class_dic

    class_train_ind = indices_save(y_train)
    class_test_ind = indices_save(y_test)

    # Definir las funciones get_batch y make_oneshot_task utilizando los datos del pliegue actual

    def get_batch(batch_size):
        # Función para generar un lote de entrenamiento
        half_batch = batch_size // 2
        pairs = [np.zeros((batch_size, genes_len, 1)) for _ in range(2)]
        targets = np.zeros((batch_size,))
        targets[half_batch:] = 1  # La segunda mitad son pares de la misma clase

        # Obtener índices de muestras de cada clase
        cancer_indices = np.where(y_train == 1)[0]
        non_cancer_indices = np.where(y_train == 0)[0]

        for i in range(half_batch):
            # Pares de clases diferentes (una cáncer, otra no cáncer)
            idx_cancer = np.random.choice(cancer_indices)
            idx_non_cancer = np.random.choice(non_cancer_indices)

            pairs[0][i, :, 0] = x_train.iloc[idx_cancer].values
            pairs[1][i, :, 0] = x_train.iloc[idx_non_cancer].values

        for i in range(half_batch, batch_size):
            # Pares de la misma clase
            if np.random.rand() < 0.5:
                # Ambos cáncer
                idx1, idx2 = np.random.choice(cancer_indices, size=2, replace=False)
            else:
                # Ambos no cáncer
                idx1, idx2 = np.random.choice(non_cancer_indices, size=2, replace=False)

            pairs[0][i, :, 0] = x_train.iloc[idx1].values
            pairs[1][i, :, 0] = x_train.iloc[idx2].values

        return pairs, targets


    def make_oneshot_task():
        # Seleccionar una muestra de prueba
        idx = np.random.choice(len(y_test))
        test_sample = x_test.iloc[idx].values.reshape(1, genes_len, 1)
        test_label = y_test[idx]

        # Crear un conjunto de soporte (support set)
        support_set = []
        targets = []
        for _ in range(N_way):
            if np.random.rand() < 0.5:
                # Muestra de la misma clase
                same_class_indices = np.where(y_test == test_label)[0]
                support_idx = np.random.choice(same_class_indices)
                targets.append(1)
            else:
                # Muestra de clase diferente
                different_class_indices = np.where(y_test != test_label)[0]
                support_idx = np.random.choice(different_class_indices)
                targets.append(0)
            support_sample = x_test.iloc[support_idx].values.reshape(1, genes_len, 1)
            support_set.append(support_sample)

        # Convertir a arrays numpy
        support_set = np.vstack(support_set)
        test_samples = np.vstack([test_sample] * N_way)
        targets = np.array(targets)

        pairs = [test_samples, support_set]

        return pairs, targets


    def test_oneshot(model, N_way, n_trials):
        n_correct = 0
        for _ in range(n_trials):
            inputs, targets = make_oneshot_task(N_way)
            probs = model.predict(inputs)
            if np.argmax(probs) == np.argmax(targets):
                n_correct += 1
        return 100.0 * n_correct / n_trials

    # Definir el modelo
    # Definir el modelo
    model = get_siamese_model(input_shape)
    optimizer = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


    # Entrenamiento del modelo
    best_acc = -1  # Reiniciar la mejor precisión para cada pliegue
    for i in range(1, n_iter + 1):
        inputs, targets = get_batch(batch_size)
        loss = model.train_on_batch(inputs, targets)

        if i % evaluate_every == 0:
            print(f"Iteración {i}, pérdida de entrenamiento: {loss}")
            val_acc = test_oneshot(model, N_way, n_val)
            print(f"Precisión de validación en one-shot ({N_way}-way): {val_acc:.2f}%")
            if val_acc >= best_acc:
                print(f"Nueva mejor precisión: {val_acc:.2f}% (anterior: {best_acc:.2f}%)")
                model.save_weights(os.path.join(model_path, f'weights_fold_{fold}.h5'))
                best_acc = val_acc

    # Evaluar en el conjunto de prueba
    test_acc = test_oneshot(model, N_way, n_val)
    print(f"Precisión en el conjunto de prueba para el pliegue {fold}: {test_acc:.2f}%")

    fold += 1


Entrenando el pliegue 1/10


ValueError: 'a' cannot be empty unless no samples are taken

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

def evaluate_model(model):
    pairs_test, targets_test = get_test_pairs()
    predictions = model.predict(pairs_test)
    predictions = predictions.ravel()

    # Calcular métricas
    acc = accuracy_score(targets_test, predictions > 0.5)
    roc_auc = roc_auc_score(targets_test, predictions)

    print(f"Precisión en el conjunto de prueba: {acc:.2f}")
    print(f"ROC-AUC en el conjunto de prueba: {roc_auc:.2f}")


In [46]:
fold=1
for train_index, test_index in kf.split(X):
    print(f"\nEntrenando el pliegue {fold}/{n_splits}")

    # Dividir los datos en entrenamiento y prueba para este pliegue
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convertir los datos a DataFrames para mantener consistencia con el código original
    x_train = pd.DataFrame(X_train, index=y_train)
    x_test = pd.DataFrame(X_test, index=y_test)

    # Crear diccionarios de índices por clase
    def indices_save(labels):
        class_dic = {}
        for k in np.unique(labels):
            indices = [i for i, label in enumerate(labels) if label == k]
            class_dic[k] = indices
        return class_dic

    class_train_ind = indices_save(y_train)
    class_test_ind = indices_save(y_test)

    # Definir las funciones get_batch y make_oneshot_task utilizando los datos del pliegue actual

    def get_batch(batch_size):
        # Función para generar un lote de entrenamiento
        categories = np.random.choice(list(class_train_ind.keys()), size=batch_size)
        pairs = [np.zeros((batch_size, genes_len, 1)) for _ in range(2)]
        targets = np.zeros((batch_size,))
        targets[batch_size // 2:] = 1

        for i in range(batch_size):
            category = categories[i]
            idx_1 = np.random.choice(class_train_ind[category])
            pairs[0][i, :, 0] = x_train.iloc[idx_1].values

            if i >= batch_size // 2:
                # Mismas clases
                idx_2 = np.random.choice(class_train_ind[category])
            else:
                # Clases diferentes
                category_2 = np.random.choice([k for k in class_train_ind.keys() if k != category])
                idx_2 = np.random.choice(class_train_ind[category_2])

            pairs[1][i, :, 0] = x_train.iloc[idx_2].values

        return pairs, targets

    def make_oneshot_task(N):
        # Función para crear una tarea de one-shot
        categories = np.random.choice(list(class_test_ind.keys()), size=N, replace=False)
        true_category = categories[0]
        indices = [np.random.choice(class_test_ind[cat]) for cat in categories]
        test_image = x_test.iloc[indices[0]].values.reshape(1, genes_len, 1)
        support_set = x_test.iloc[indices].values.reshape(N, genes_len, 1)

        test_images = np.vstack([test_image] * N)
        targets = np.zeros((N,))
        targets[0] = 1  # La primera es la correcta

        # Mezclar el soporte y las etiquetas
        pairs = [test_images, support_set]
        targets, pairs[0], pairs[1] = shuffle(targets, pairs[0], pairs[1])

        return pairs, targets

    def test_oneshot(model, N_way, n_trials):
        n_correct = 0
        for _ in range(n_trials):
            inputs, targets = make_oneshot_task(N_way)
            probs = model.predict(inputs)
            if np.argmax(probs) == np.argmax(targets):
                n_correct += 1
        return 100.0 * n_correct / n_trials

    # Definir el modelo
    model = get_siamese_model(input_shape)
    optimizer = Adam(lr=0.000005)
    model.compile(loss="binary_crossentropy", optimizer=optimizer)

    # Entrenamiento del modelo
    best_acc = -1  # Reiniciar la mejor precisión para cada pliegue
    for i in range(1, n_iter + 1):
        inputs, targets = get_batch(batch_size)
        loss = model.train_on_batch(inputs, targets)

        if i % evaluate_every == 0:
            print(f"Iteración {i}, pérdida de entrenamiento: {loss}")
            val_acc = test_oneshot(model, N_way, n_val)
            print(f"Precisión de validación en one-shot ({N_way}-way): {val_acc:.2f}%")
            if val_acc >= best_acc:
                print(f"Nueva mejor precisión: {val_acc:.2f}% (anterior: {best_acc:.2f}%)")
                model.save_weights(os.path.join(model_path, f'weights_fold_{fold}.h5'))
                best_acc = val_acc

    # Evaluar en el conjunto de prueba
    test_acc = test_oneshot(model, N_way, n_val)
    print(f"Precisión en el conjunto de prueba para el pliegue {fold}: {test_acc:.2f}%")

    fold += 1


Entrenando el pliegue 1/10


/usr/lib/python3/dist-packages/keras/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Iteración 100, pérdida de entrenamiento: 0.72700434923172


ValueError: Cannot take a larger sample than population when 'replace=False'